<a href="https://colab.research.google.com/github/buketugurlu/BUSI/blob/main/BUSI_inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install segmentation-models-pytorch
!pip install torch torchvision
!pip install scikit-learn

# Import libraries
import os
from google.colab import drive
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import segmentation_models_pytorch as smp
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

# Dataset directory
base_dir = '/content/drive/My Drive/Dataset_BUSI_with_GT'


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.8 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=fefc8788defa52edd3684f2164961e77d9de14e35d8e8b415298ff8abb0f0f07
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=8cbb05b4a8901b9f0cba4d67a1fbbd8f8dd3a03cba206ef4f5450c72730972cd
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorc

In [7]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import inception_v3, Inception_V3_Weights
from PIL import Image, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt

# Custom Dataset Class
class BreastUltrasoundDataset(Dataset):
    def __init__(self, root_dir, classes=['benign', 'malignant'], transform_image=None, transform_mask=None):
        self.root_dir = root_dir
        self.classes = classes
        self.transform_image = transform_image
        self.transform_mask = transform_mask
        self.image_paths = []
        self.mask_paths = []

        # Load image and mask paths
        for cls in self.classes:
            image_dir = os.path.join(root_dir, cls)
            for filename in os.listdir(image_dir):
                if filename.endswith(".png") and "_mask" not in filename:
                    image_path = os.path.join(image_dir, filename)
                    mask_path = os.path.join(image_dir, filename.replace(".png", "_mask.png"))
                    self.image_paths.append(image_path)
                    self.mask_paths.append(mask_path)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        mask = Image.open(self.mask_paths[idx]).convert("L")  # Grayscale mask

        # Apply data augmentations
        if np.random.rand() > 0.5:
            image = ImageEnhance.Contrast(image).enhance(1.5)  # Increase contrast
        if np.random.rand() > 0.5:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)  # Horizontal flip
            mask = mask.transpose(Image.FLIP_LEFT_RIGHT)
        if np.random.rand() > 0.5:
            image = image.transpose(Image.FLIP_TOP_BOTTOM)  # Vertical flip
            mask = mask.transpose(Image.FLIP_TOP_BOTTOM)

        # Apply transformations
        if self.transform_image:
            image = self.transform_image(image)
        if self.transform_mask:
            mask = self.transform_mask(mask)

        return image, mask

# Image and mask transformations
transform_image = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

transform_mask = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Create dataset and dataloader
dataset = BreastUltrasoundDataset(
    root_dir=base_dir,
    classes=['benign', 'malignant'],
    transform_image=transform_image,
    transform_mask=transform_mask
)

train_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

# Custom InceptionV3-based segmentation model
class InceptionSegmentationModel(nn.Module):
    def __init__(self, weights=Inception_V3_Weights.IMAGENET1K_V1, num_classes=1):
        super(InceptionSegmentationModel, self).__init__()
        # Load pretrained InceptionV3
        self.encoder = inception_v3(weights=weights, aux_logits=True)
        self.encoder.aux_logits = False  # Disable auxiliary logits in forward pass
        self.encoder.avgpool = nn.Identity()  # Remove avgpool to preserve spatial dimensions
        self.encoder.fc = nn.Identity()  # Remove classification head

        # Decoder head
        self.decoder = nn.Sequential(
            nn.Conv2d(2048, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True),
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=4, mode="bilinear", align_corners=True),  # Upsample to match input size
            nn.Conv2d(64, num_classes, kernel_size=1),  # Final output
        )

    def forward(self, x):
        x = F.interpolate(x, size=(299, 299), mode="bilinear", align_corners=False)  # Resize for InceptionV3
        features = self.encoder(x)  # Get encoder features (preserve spatial dimensions)

        # Check if features are a tuple (auxiliary logits present)
        if isinstance(features, tuple):
            features = features[0]  # Use the main feature map

        # Ensure the output from encoder is in the correct shape
        if len(features.shape) == 2:  # Flattened tensor [batch_size, 131072]
            batch_size = features.size(0)
            features = features.view(batch_size, 2048, 8, 8)  # Reshape to spatial dimensions

        return self.decoder(features)  # Decode features for segmentation


# Model initialization
model = InceptionSegmentationModel(weights=Inception_V3_Weights.DEFAULT, num_classes=1).cuda()

# Modeli başlat
model = InceptionSegmentationModel(weights=Inception_V3_Weights.DEFAULT, num_classes=1).cuda()

# Initialize the model
model = InceptionSegmentationModel(weights=Inception_V3_Weights.DEFAULT, num_classes=1).cuda()

# Loss function and optimizer
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, masks in train_loader:
        images, masks = images.cuda(), masks.cuda()
        optimizer.zero_grad()
        predictions = model(images)
        loss = loss_function(predictions, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), '/content/drive/My Drive/Dataset_BUSI_with_GT/inception_segmentation_model.pth')


Epoch 1/20, Loss: 0.32211056060907317
Epoch 2/20, Loss: 0.19297192336582555
Epoch 3/20, Loss: 0.15879214282442883
Epoch 4/20, Loss: 0.13235575651250234
Epoch 5/20, Loss: 0.11439458535211842
Epoch 6/20, Loss: 0.1011086337086631
Epoch 7/20, Loss: 0.0911369012078134
Epoch 8/20, Loss: 0.083840618954926
Epoch 9/20, Loss: 0.07840755627286143
Epoch 10/20, Loss: 0.07878207987764986
Epoch 11/20, Loss: 0.06868254738609965
Epoch 12/20, Loss: 0.0679053499749521
Epoch 13/20, Loss: 0.0640453382781366
Epoch 14/20, Loss: 0.059566287914427314
Epoch 15/20, Loss: 0.055132332462363126
Epoch 16/20, Loss: 0.05369110205551473
Epoch 17/20, Loss: 0.04993223626075721
Epoch 18/20, Loss: 0.0480206675554921
Epoch 19/20, Loss: 0.05955943465232849
Epoch 20/20, Loss: 0.053341415869753536


In [8]:
# PyTorch ve torchvision (model ve veri işlemleri için)
!pip install torch torchvision

# scikit-learn (accuracy, precision, recall, F1-score gibi metrikler için)
!pip install scikit-learn

# PIL (Python Imaging Library) ve NumPy (görüntü işleme ve tensor manipülasyonu için)
!pip install pillow numpy

# Matplotlib (görselleştirme için)
!pip install matplotlib


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(dataset, model, num_samples=100):
    """
    Evaluate the segmentation model on a dataset and compute metrics.

    Args:
    - dataset: The dataset to evaluate on.
    - model: The trained segmentation model.
    - num_samples: Number of samples to evaluate (default is 100).

    Returns:
    - Prints accuracy, precision, recall, F1 score, Dice score, and IoU.
    """
    model.eval()
    all_preds = []
    all_masks = []

    with torch.no_grad():
        for i in range(min(num_samples, len(dataset))):
            image, mask = dataset[i]
            image = image.unsqueeze(0).cuda()  # Add batch dimension
            mask = mask.squeeze().cpu().numpy()  # Ground truth mask in NumPy format

            # Get model prediction
            prediction = model(image)
            prediction = torch.sigmoid(prediction).cpu().squeeze().numpy()  # Apply sigmoid activation

            # Binarize the prediction with a threshold of 0.5
            prediction_binary = (prediction > 0.5).astype(np.uint8)

            # Flatten the arrays for metric computation
            all_preds.append(prediction_binary.flatten())
            all_masks.append(mask.flatten())

    # Concatenate all predictions and masks
    y_pred = np.concatenate(all_preds)
    y_true = np.concatenate(all_masks).astype(np.uint8)

    # Compute metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=1)
    recall = recall_score(y_true, y_pred, zero_division=1)
    f1 = f1_score(y_true, y_pred, zero_division=1)

    # Dice score
    intersection = np.logical_and(y_true, y_pred).sum()
    dice_score = (2.0 * intersection) / (y_true.sum() + y_pred.sum() + 1e-6)

    # IoU (Jaccard Index)
    union = np.logical_or(y_true, y_pred).sum()
    iou = intersection / (union + 1e-6)

    # Print metrics
    print("Evaluation Metrics:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1 Score: {f1:.4f}")
    print(f"  Dice Score: {dice_score:.4f}")
    print(f"  IoU (Jaccard Index): {iou:.4f}")


In [10]:
evaluate_model(dataset, model, num_samples=len(dataset))  # Tüm dataset için


Evaluation Metrics:
  Accuracy: 0.9812
  Precision: 0.8799
  Recall: 0.9142
  F1 Score: 0.8968
  Dice Score: 0.8968
  IoU (Jaccard Index): 0.8128
